In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

#### 用convert_dtypes()读取数据

In [2]:
train = pd.read_csv('./input/train.csv').convert_dtypes(convert_string=False)
test = pd.read_csv('./input/testA.csv').convert_dtypes(convert_string=False)

#### 定义全局变量

# 先拼train和test 本来想封装一个函数用来拼接
因为似乎concat有个问题，会lose dtype?
复盘了一下M5 感觉M5-simple-fe，def merge-by-concat不太看得懂

In [3]:
# train.shape  # (800000, 48) & len(train)=800000

In [4]:
# test.shape # (200000, 47)

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [9]:
TARGET = 'isDefault'
train['flag'] = 'train'
test['flag'] = 'test'
test[TARGET] = -1
df = pd.concat([train, test])
df

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,flag
0,0,35000,5,19.52,917.97,E,E2,320,2 years,2,110000.0,2,2014-07-01,1,1,137,32,17.05,0,730,734,7,0,0,24178,48.9,27,0,0,Aug-2001,1,1,0,2,2,2,4,9,8,4,12,2,7,0,0,0,2,train
1,1,18000,5,18.49,461.90,D,D2,219843,5 years,0,46000.0,2,2012-08-01,0,0,156,18,27.83,0,700,704,13,0,0,15096,38.9,18,1,0,May-2002,1723,1,<NA>,<NA>,<NA>,<NA>,10,<NA>,<NA>,<NA>,<NA>,<NA>,13,<NA>,<NA>,<NA>,<NA>,train
2,2,12000,5,16.99,298.17,D,D3,31698,8 years,0,74000.0,2,2015-10-01,0,0,337,14,22.77,0,675,679,11,0,0,4606,51.8,27,0,0,May-2006,0,1,0,0,3,3,0,0,21,4,5,3,11,0,0,0,4,train
3,3,11000,3,7.26,340.96,A,A4,46854,10+ years,1,118000.0,1,2015-08-01,0,4,148,11,17.21,0,685,689,9,0,0,9948,52.6,28,1,0,May-1999,4,1,6,4,6,6,4,16,4,7,21,6,9,0,0,0,1,train
4,4,3000,3,12.99,101.07,C,C2,54,NaN,1,29000.0,2,2016-03-01,0,10,301,21,32.16,0,690,694,12,0,0,2942,32.0,27,0,0,Aug-1977,11,1,1,2,7,7,2,4,9,10,15,7,12,0,0,0,4,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,999995,7000,3,11.14,229.64,B,B2,330967,7 years,1,30000.0,2,2012-10-01,-1,4,36,13,32.92,0,685,689,8,0,0,5021,54.0,17,1,0,Nov-2005,52663,1,0,2,2,2,2,6,11,2,6,2,8,0,0,0,4,test
199996,999996,6000,3,6.24,183.19,A,A2,38930,1 year,1,56000.0,0,2015-10-01,-1,0,261,8,20.38,0,720,724,25,0,0,10163,35.9,27,0,0,Oct-2006,0,1,0,5,6,6,5,5,14,12,13,6,25,0,0,0,0,test
199997,999997,14000,5,15.88,339.57,C,C4,282016,8 years,2,80000.0,2,2013-07-01,-1,4,46,27,15.12,0,675,679,21,0,0,31401,86.0,63,0,0,Dec-2001,4,1,0,7,13,13,9,21,18,21,42,13,21,0,0,0,0,test
199998,999998,8000,3,18.06,289.47,D,D2,97,4 years,1,190000.0,0,2017-10-01,-1,0,99,8,9.33,0,690,694,11,0,0,28493,64.0,24,0,0,Aug-2005,0,1,0,3,6,6,4,12,5,8,19,6,11,0,0,0,2,test


In [ ]:
train[col] = df[: len(train)]
test[col] = df[len(train):]
 
df = pd.concat([train[col], test[col], axis = 0])

# drop policycode

In [ ]:
# train.dtypes Int

#### issueDate

#### 这里对了吗？？？datetime? string?

In [ ]:
#转化成时间格式，查了确实train的最小日期是2007-6-1，这里加了个issueDateDT,还行吧，就是日期会让数值特别大，
for data in [train, testa]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    # 构造时间特征
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [ ]:
# train.dtypes   # issueDate datetime64[ns]

#### employmentLength的 years删掉，10+ <1转

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [train, testa]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [ ]:
train['employmentLength'].value_counts(dropna=False).sort_index()

#### 这里float，转成Int8，就不知道这种用祖传函数能不能自动转?

In [ ]:
for data in [train, testa]:
    data['employmentTitle'] = data['employmentTitle'].astype('Int8')

#### earliesCreditLine 这个正常了，就是值有点多，不知道要怎么利用抽取信息做特征

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
train['earliesCreditLine'].sample(20)

In [ ]:
for data in [train, testa]:
    data['earliesCreditLine'] = pd.to_datetime(data['earliesCreditLine'],format='%b-%Y')

In [ ]:
pd.set_option('display.max_columns', None)
train.describe(include="all")

In [ ]:
train.info()

In [ ]:
# dw这里是提取年信息，我想利用原始信息，顺便还可以describe一下,暂时不用他的
# for data in [train, testa]:
#     data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

#### 压缩一下内存

In [ ]:
import psutil

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [ ]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['Int16', 'Int32', 'Int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'Int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype('Int8')
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype('Int16')
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype('Int32')
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype('Int64')
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

这里可以拼train 和test成df

In [ ]:
print("{:>20}: {:>8}".format('Original df',sizeof_fmt(train.memory_usage(index=True).sum())))

In [ ]:
train = reduce_mem_usage(train)

In [ ]:
train.info()

#### 数据类型转换

In [ ]:
# cat_fea = ['grade', 'subGrade', 'employmentTitle', 'verificationStatus', \
#           'purpose', 'postCode', 'regionCode','applicationType','initialListStatus', 'title', 'policyCode']

In [ ]:
# cols = ['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
#        'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
#        'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
#        'purpose', 'postCode', 'regionCode', 'dti', 'detlinquency_2years',
#        'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
#        'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
#        'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
#        'policyCode', 'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
#        'n10', 'n11', 'n12', 'n13', 'n14']

'issueDate', 'isDefault', 'earliesCreditLine'

In [ ]:
# num_fea = [a for a in cols if a not in cat_fea]
# num_fea

In [ ]:
# ## 定义了一个统计函数，方便后续信息统计
# def Sta_inf(data):
#     print('_min',np.min(data))
#     print('_max:',np.max(data))
#     print('_mean',np.mean(data))
#     print('_ptp',np.ptp(data))
#     print('_std',np.std(data))
#     print('_var',np.var(data))